In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('data/diabetes_data.csv')

In [2]:
dup = data.duplicated(subset=list(data.columns))
clean = data.drop_duplicates(subset=list(data.columns))

In [ ]:
lst = []

for col in data.columns:
    top_freq = data[col].value_counts(normalize=True).max()
    nunique_ratio = data[col].nunique() / data[col].count()
    if top_freq > 0.95:
        lst.append(col)
    if nunique_ratio > 0.95:
        lst.append(col)
        
clean = clean.drop(lst, axis=1)

In [20]:
clean[['Glucose', 'Insulin', 'BloodPressure', 'SkinThickness', 'BMI']] = clean[
    ['Glucose', 'Insulin', 'BloodPressure', 'SkinThickness', 'BMI']
    ].replace(to_replace=0, value=np.nan)

In [30]:
dro = clean.copy()
thresh = dro.shape[0]*0.7
dro = dro.dropna(thresh=thresh, axis=1)
m = dro.shape[1]
dro = dro.dropna(thresh=m-2, axis=0)

In [ ]:
dic = {'Glucose': dro['Glucose'].median(),
       'BloodPressure': dro['BloodPressure'].median(),
       'SkinThickness': dro['SkinThickness'].median(),
       'BMI': dro['BMI'].median(),
       }
clean = dro.fillna(dic)
clean['SkinThickness'].mean()

In [43]:
def outliers_iqr(data, feature, left=1.5, right=1.5, log_scale=True):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(clean, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 0
Результирующее число записей: 761


In [40]:
def outliers_z_score(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(clean, 'SkinThickness', log_scale=False)

In [41]:
outliers

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
30,0,162.0,76.0,56.0,53.2,0.759,25,1
36,2,197.0,70.0,99.0,34.7,0.575,62,1
382,0,180.0,78.0,63.0,59.4,2.420,25,1
444,0,100.0,88.0,60.0,46.8,0.962,31,0


In [ ]:
29